### 1. DEFINICION DEL MODELO A UTILIZAR

In [ ]:
# define model
MODEL_NAME = "openai-community/gpt2-medium"

### 2. CARGUE DE LIBRERIAS DE FORMATO Y CONFIGURACIONES DE JUPYTER BLACK

In [ ]:
# Formatting libraries
!pip install black jupyter-black
import black
import jupyter_black

# Load jupyter_black settings
jupyter_black.load(
    lab=True,
    line_length=170,
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 3.5 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.7 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


### 3. CARGUE Y PREPARACION DEL DATASET


In [ ]:
!pip install datasets
from datasets import load_dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!pip install py7zr
dataset = load_dataset("samsum")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.8/413.8 kB 32.9 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/samsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

### 4. DEFINIR EL TOKENIZADOR

In [ ]:

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, padding_side="right")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

### 5. DEFINICION DE LA PLANTILLA Y CARGUE DEL PROMT

In [ ]:
# define prompt template
prompt_template = """
Summarize the following conversation.

### Conversation:

{dialogue}

### Summary:

"""


# create prompt
def create_prompt(data):
    dialogue = data["dialogue"]
    summary = data["summary"]
    prompt = prompt_template.format(dialogue=dialogue, summary=summary)

    n_tokens_output = len(tokenizer.encode(summary, add_special_tokens=False))
    n_tokens_input = len(tokenizer.encode(prompt, add_special_tokens=False))

    return {"input": prompt, "output": summary, "n_tokens_input": n_tokens_input, "n_tokens_output": n_tokens_output}

In [ ]:
dataset = dataset.map(create_prompt)
dataset

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1100 > 1024). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input', 'output', 'n_tokens_input', 'n_tokens_output'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input', 'output', 'n_tokens_input', 'n_tokens_output'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input', 'output', 'n_tokens_input', 'n_tokens_output'],
        num_rows: 818
    })
})

### 6. CREACION DE PERCENTILES Y CUARTILES PARA LA LONGITUD DE LOS TOKENS

In [ ]:
# get 0.95 percentile of dialogue length in training set
dataset["train"].to_pandas().n_tokens_input.quantile(0.98)

482.0

In [ ]:
# get 0.95 quantile of n_tokens_summary in train dataset`
dataset["train"].to_pandas().n_tokens_output.quantile(0.98)

62.0

In [ ]:
# filter very long dialogs and summaries
dataset = dataset.filter(lambda x: x["n_tokens_input"] < 480 and x["n_tokens_output"] < 60)

Filter:   0%|          | 0/14732 [00:00<?, ? examples/s]

Filter:   0%|          | 0/819 [00:00<?, ? examples/s]

Filter:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
print(dataset["train"]["input"][0])


Summarize the following conversation.

### Conversation:

Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-)

### Summary:




In [ ]:
print(dataset["train"]["output"][0])

Amanda baked cookies and will bring Jerry some tomorrow.


### 7. CARGUE Y PREPARACION DEL TOKENIZADOR

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, add_special_tokens=False)

# verify the existing special tokens
print(f"Special Tokens: \n{tokenizer.special_tokens_map}")

# if no padding token set eos_token as padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

Special Tokens: 
{'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}


In [ ]:
# load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="cuda")

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

### 8. DEFINICION DEL DATALOADER

In [ ]:
# import pad_sequence from torch
from torch.nn.utils.rnn import pad_sequence
import torch
import copy

In [ ]:
class DataCollatorForCausalLM:
    def __init__(self, tokenizer, source_max_len, target_max_len, training_on_source, padding_side):
        self.tokenizer = tokenizer
        self.source_max_len = source_max_len
        self.target_max_len = target_max_len
        self.training_on_source = training_on_source
        self.padding_side = padding_side

    def __call__(self, instances):
        source = [tokenizer.bos_token + text["input"] for text in instances]
        target = [text["output"] + tokenizer.eos_token for text in instances]

        source = tokenizer(
            source,
            padding=False,
            truncation=True,
            max_length=self.source_max_len,
        )

        target = tokenizer(
            target,
            padding=False,
            truncation=True,
            max_length=self.target_max_len,
        )

        input_ids = []
        labels = []
        for source_tokens, target_token in zip(source["input_ids"], target["input_ids"]):
            if tokenizer.pad_token_id == tokenizer.eos_token_id:
                target_token[-1] = -50

            input_ids.append(torch.LongTensor(source_tokens + target_token))

            if self.training_on_source:
                labels.append(torch.LongTensor(copy.deepcopy(source_tokens + target_token)))
            else:
                labels.append(torch.LongTensor([-100] * len(source_tokens) + copy.deepcopy(target_token)))

        # Pad sequences to the longest one in the batch
        if self.padding_side == "right":
            input_ids = pad_sequence(input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id)
            labels = pad_sequence(labels, batch_first=True, padding_value=-100)

        elif self.padding_side == "left":
            # Find the maximum sequence length in the batch
            max_length = max(len(seq) for seq in input_ids)

            # Perform left padding manually
            input_ids = [torch.cat([torch.full((max_length - len(seq),), self.tokenizer.pad_token_id, dtype=torch.long), seq]) for seq in input_ids]
            labels = [torch.cat([torch.full((max_length - len(seq),), -100, dtype=torch.long), seq]) for seq in labels]

            # Convert list of tensors to a single tensor
            input_ids = torch.stack(input_ids)
            labels = torch.stack(labels)

        else:
            raise ValueError("padding_direction must be 'right' or 'left'")

        attention_mask = input_ids.ne(self.tokenizer.pad_token_id)

        if tokenizer.pad_token_id == tokenizer.eos_token_id:
            input_ids[input_ids == -50] = self.tokenizer.pad_token_id
            labels[labels == -50] = self.tokenizer.pad_token_id

        return {"input_ids": input_ids, "labels": labels, "attention_mask": attention_mask}

### 9. ENTRENAMIENTO DEL MODELO

In [ ]:
# Define collator
data_collator = DataCollatorForCausalLM(
    tokenizer=tokenizer,
    source_max_len=480,
    target_max_len=60,
    training_on_source=False,
    padding_side="left",
)

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
# Define the training arguments for the Trainer
training_arguments = TrainingArguments(
    report_to="tensorboard",
    logging_dir=f"./{MODEL_NAME}-tensorboard",
    output_dir=f"./{MODEL_NAME}-checkpoing",
    num_train_epochs=5,
    optim="adamw_torch",
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    gradient_accumulation_steps=5,
    max_steps=-1,
    weight_decay=0.01,
    learning_rate=5e-5,
    remove_unused_columns=False,
    max_grad_norm=1.0,
    gradient_checkpointing=True,
    do_train=True,
    do_eval=True,
    lr_scheduler_type="constant",
    warmup_ratio=0.005,
    logging_strategy="epoch",
    logging_steps=100,
    evaluation_strategy="epoch",
    eval_steps=100,  # We use same value for logging_steps (train metrics) and  eval_steps (eval metrics).
    group_by_length=False,
    save_strategy="epoch",
    save_steps=100,
    save_total_limit=100,
    load_best_model_at_end=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Initialize the Trainer with the model, tokenizer, training arguments, datasets, and data collator
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset["train"].shuffle(seed=42),#.select(range(5000)),
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
)

# Start the training process
trainer.train()

# Save the trained model to the specified path
trainer.model.save_pretrained(f"./{MODEL_NAME}-fine-tuned-model")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,2.001600,1.534434
1,1.429800,1.494818
2,1.229300,1.501496
3,1.070300,1.545876
4,0.917100,1.620981


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning

In [ ]:
trainer